In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('kaggle/input/robi-datathon-30-qualifier-problem-1/purchase.csv')
db = pd.read_csv('kaggle/input/robi-datathon-30-qualifier-problem-1/boxes.csv')

4. How many records have missing information?

In [5]:
df_null = df[df.isnull().any(axis=1)]
print(df_null)
print(len(df_null))

        PURCHASE_DATE    MAGIC_KEY  BOX_ID  BOX_COUNT
177          1/2/2019  28FF265F082     NaN        NaN
178          1/2/2019  2CB168CDFA3     NaN        NaN
179          1/2/2019  2C2C8844F09     NaN        NaN
180          1/2/2019  2BDCCEF05A4     NaN        NaN
181          1/2/2019  2BF1D98D0B2     NaN        NaN
182          1/2/2019  2BDC3619EAF     NaN        NaN
183          1/2/2019  290D33249B7     NaN        NaN
184          1/2/2019  2BE266F8C55     NaN        NaN
185          1/2/2019  2903192D056     NaN        NaN
186          1/2/2019  2BE34886311     NaN        NaN
187          1/2/2019  29E5BBC4C1C     NaN        NaN
188          1/2/2019  2C5D80EF887     NaN        NaN
189          1/2/2019  2CE917B8F56     NaN        NaN
190          1/2/2019  2C0BB928475     NaN        NaN
235979      12/2/2019  2CB168CDFA3     NaN        NaN
379928      20/2/2019  2C2C8844F09     NaN        NaN
380268      20/2/2019  290D33249B7     NaN        NaN
399457      21/2/2019  2BDC3

5. How many records are invalid?

In [11]:
df_invalid = df[df.BOX_COUNT <= 0 | df.BOX_ID.isnull() | (df.BOX_ID > 290)]
print(df_invalid)
print(len(df_invalid))

        PURCHASE_DATE    MAGIC_KEY      BOX_ID  BOX_COUNT
191          1/2/2019  2CCB122460D  11111111.0       -1.0
192          1/2/2019  2CEDA71E3E9  11111111.0       -1.0
193          1/2/2019  2CBDC970258  11111111.0       -1.0
194          1/2/2019  2932761FEB6  11111111.0       -1.0
195          1/2/2019  28F09714410  11111111.0       -1.0
196          1/2/2019  2BDAE37A00E  11111111.0       -1.0
197          1/2/2019  2C3C7E168F7  11111111.0       -1.0
525713      27/2/2019  2C3C7E168F7  11111111.0       -1.0
597615       1/1/2019  2CEDA71E3E9  11111111.0       -1.0
665861       5/1/2019  2C3C7E168F7  11111111.0       -1.0
688095       6/1/2019  2CBDC970258  11111111.0       -1.0
1251257    10/12/2018  2C3C7E168F7  11111111.0       -1.0
1834669    10/11/2018  2C3C7E168F7  11111111.0       -1.0
1943916    24/11/2018  2CBDC970258  11111111.0       -1.0
2002908     5/10/2018  2CBDC970258  11111111.0       -1.0
2105309    13/10/2018  2BDAE37A00E  11111111.0       -1.0
2123428    12/

6. What is the monthly meat sales quantity for Feb'19? 

In [19]:
df['PURCHASE_DATE'] = pd.to_datetime(df['PURCHASE_DATE'], format='%d/%m/%Y')
df_feb = df[(df['PURCHASE_DATE'].dt.year == 2019) & (df['PURCHASE_DATE'].dt.month == 2)]
dc = df_feb.merge(db, on="BOX_ID", how="inner")
dc['MUL'] = dc['BOX_COUNT'] * dc['MEAT']
print(dc['MUL'].sum())

1272671.6


7. What is the Month-on-Month meat sales amount growth% on average?

In [20]:
ar = []
dates = [(2018, 10), (2018, 11), (2018, 12), (2019, 1), (2019, 2)]
for i, j in dates:
    df1 = df[(df['PURCHASE_DATE'].dt.year == i) & (df['PURCHASE_DATE'].dt.month == j)]
    dc = df1.merge(db, on="BOX_ID", how="inner")
    dc['QUANTITY'] = dc['BOX_COUNT'] * dc['MEAT']
    ar.append(dc['QUANTITY'].sum())
print(ar)

[814079.1, 851061.6000000001, 1002901.6000000002, 1113503.1000000003, 1272671.6]


In [21]:
grate = []
for i in range(1, len(ar)):
    rate = ((ar[i] - ar[i-1])/ar[i-1]) * 100
    grate.append(rate)
print(grate)
average = sum(grate) / len(grate)
print(average)

[4.542863218082877, 17.84124674406648, 11.028150717877018, 14.294392175468548]
11.92666321387373


8. What is the difference between Daily avg. Milk sale amount on Sunday & Tuesday for the month of Jan'19?

In [22]:
df_jan = df[(df['PURCHASE_DATE'].dt.year == 2019) & (df['PURCHASE_DATE'].dt.month == 1)]
dc = df_jan.merge(db, on="BOX_ID", how="inner")
dc['QUANTITY'] = dc['BOX_COUNT'] * dc['MILK']
# Monday = 0, Sunday = 6
ds = dc[dc['PURCHASE_DATE'].dt.dayofweek == 6]
dt = dc[dc['PURCHASE_DATE'].dt.dayofweek == 1]
s_mean = ds['QUANTITY'].mean()
t_mean = dt['QUANTITY'].mean()
print(s_mean - t_mean)

-0.11323648653925034


9. What is the top selling premium quality box ID for the month of Nov'18?

In [32]:
df_nov_18 = df[(df['PURCHASE_DATE'].dt.year == 2018) & (df['PURCHASE_DATE'].dt.month == 11)]
dc = df_nov_18.merge(db, on="BOX_ID", how="inner")
dc = dc[dc['QUALITY'] == "Premium"]
dg = dc.groupby('BOX_ID')
dg.head()
index = dg['BOX_COUNT'].sum().idxmax()
print(index)

163.0


10. What is the unit price of the third highest sold box for Dec'18?

In [34]:
df_dec_18 = df[(df['PURCHASE_DATE'].dt.year == 2018) & (df['PURCHASE_DATE'].dt.month == 12)]
dc = df_dec_18.merge(db, on="BOX_ID", how="inner")
# Group by 'BOX_ID' and sum the 'BOX_COUNT', also retrieve the 'UNIT_PRICE' (assumed constant per box id)
grouped_data = dc.groupby('BOX_ID').agg(Total_Sold=('BOX_COUNT', 'sum'), Unit_Price=('UNIT_PRICE', 'first')).reset_index()

# Sort the data by 'Total_Sold' in descending order
sorted_data = grouped_data.sort_values(by='Total_Sold', ascending=False)

# Get the third highest sold box's unit price
# Check if there are at least three boxes
third_highest_sold_box_price = sorted_data.iloc[2]['Unit_Price']
print(third_highest_sold_box_price)

15.96
